In [ ]:
#%%

import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [ ]:
# %%

df = pd.read_csv("archive/blogtext.csv")

In [ ]:
# %%

# Clean the text data
def clean_text(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove non-alphanumeric characters and extra spaces
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
# %%

df['clean_text'] = df['text'].apply(clean_text)

In [ ]:
# %%

# Encode the gender column
label_encoder = LabelEncoder()
df['gender_encoded'] = label_encoder.fit_transform(df['gender'])  # Male: 0, Female: 1

In [ ]:
# %%

# Train-test split
X = df['clean_text']  # Features (text data)
y = df['gender_encoded']  # Target (encoded gender)

In [ ]:
# %%

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
#%%

device = ('cuda' if torch.cuda.is_available() else 'cpu')

NameError: name 'torch' is not defined

In [ ]:
#%%

from transformers import DistilBertTokenizer, DistilBertModel
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from tqdm import tqdm


device = ('cuda' if torch.cuda.is_available() else 'cpu')

# Load pretrained DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
model = model.to(device)
model.eval()

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [ ]:
# %%

def generate_embeddings(texts, tokenizer, model, max_length=128):
    embeddings = []
    for text in tqdm(texts, desc="Generating Embeddings"):
        # Tokenize text
        tokens = tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        input_ids = tokens['input_ids'].to(device)
        attention_mask = tokens['attention_mask'].to(device)

        # Pass through model
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            token_embeddings = outputs.last_hidden_state  # Shape: [batch_size, sequence_length, hidden_size]
            sentence_embedding = torch.mean(token_embeddings, dim=1)  # Mean pooling over tokens
            embeddings.append(sentence_embedding.cpu().numpy())
    
    return torch.cat(embeddings, dim=0)  # Return as a single tensor

# Generate embeddings for training and testing datasets
X_train_embeddings = generate_embeddings(X_train.tolist(), tokenizer, model)
X_test_embeddings = generate_embeddings(X_test.tolist(), tokenizer, model)

Generating Embeddings: 100%|██████████| 545027/545027 [1:27:36<00:00, 103.69it/s] 


TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray

In [ ]:
# %%

# Flatten embeddings for classifier input
X_train_embeddings = X_train_embeddings.squeeze()
X_test_embeddings = X_test_embeddings.squeeze()

# Train Logistic Regression on the extracted embeddings
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_embeddings, y_train)

# Predict on the test set
y_pred = log_reg.predict(X_test_embeddings)

# Evaluate the classifier
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

: 

In [ ]:
# %%

# Flatten embeddings for classifier input
X_train_embeddings = X_train_embeddings.squeeze()
X_test_embeddings = X_test_embeddings.squeeze()

# Train Logistic Regression on the extracted embeddings
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_embeddings, y_train)

# Predict on the test set
y_pred = log_reg.predict(X_test_embeddings)

# Evaluate the classifier
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

NameError: name 'X_train_embeddings' is not defined